In [85]:
# Package imports

import matplotlib.pyplot as plt
import numpy as np
import os 
import sys
from PIL import Image
import random
import cv2
import functools
from copy import deepcopy

In [86]:
height = 64
width = 64
original = os.path.join('dataset','photos')
new = os.path.join('dataset','compressed')
files=os.listdir(original)

In [87]:
#resize images 
for file in files:    
    path =  os.path.join(original,file)
    new_path = os.path.join(new, file)
    image = Image.open(path)
    image = image.resize((height,width), Image.ANTIALIAS)   
    image.save(new_path)

In [88]:
# get all images from the reduced dataset, and construct the dataset
Y = [] # output
Yid = {} # name to number Y_values
Yname = {} # nuber to name Y_inverse_values 

X = [] # training dataset R
files=os.listdir(new)

for file in files:    
    # for X
    image_path=os.path.join(new, file)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image.flatten()
    X.append(image)
    # for Y
    name = file.split("_")[0]
    if(name not in Yid) :
        val = len(Yid) + 1
        Yid[name] = val
        Yname[val] = name
    Y.append(Yid[name])
        
X = np.array(X)
Y = np.array(Y)

In [89]:
#mean_intensities
u = np.mean(X, axis=0)

In [90]:
# split into training and testing data
X_train = []
X_test = []
Y_train = []
Y_test = []

percent = 80 
sample_size = int((percent / float(100)) * X.shape[0])
training_indices = random.sample(range(0, X.shape[0]), sample_size)
for i in range(X.shape[0]) :
    if(i in training_indices) :
        X_train.append(X[i])
        Y_train.append(Y[i])
    else :
        X_test.append(X[i])
        Y_test.append(Y[i])
        
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [91]:
#noralising training dataset
utrain = np.mean(X_train,axis=0)
X_train = X_train - utrain

In [92]:
#normlaising testing dataset
X_test_original = deepcopy(X_test)
utest = np.mean(X_test,axis=0)
X_test = X_test - utest

In [93]:
# finding covariance matrix 
covariance = (1 / float(X_train.shape[0])) * np.dot(np.transpose(X_train), X_train)

In [94]:
# finding best K values
u, s, v = np.linalg.svd(covariance)
totalsum = 0
for i in range(s.shape[0]) :
    totalsum = totalsum + s[i]
sm = 0
for i in range(s.shape[0]) :
    sm = sm + s[i]
    val = float(sm) / float(totalsum)
    if(val >= 0.99) :
        K=i+1
        break

In [95]:
#calculating eigen vectors
EigenVectors = []
for i in range(K) :
    EigenVectors.append(u[:,i])
EigenVectors = np.array(EigenVectors)

In [96]:
EigenVectors

array([[-0.00571757, -0.00534319, -0.00443315, ...,  0.0123138 ,
         0.01218299,  0.01124567],
       [-0.00054723, -0.00071774, -0.00168021, ...,  0.01159185,
         0.01250289,  0.01012099],
       [ 0.0290654 ,  0.0288687 ,  0.02922227, ..., -0.02679854,
        -0.02789048, -0.02718663],
       ...,
       [ 0.01922902,  0.006195  , -0.0032372 , ...,  0.0301986 ,
         0.00287601, -0.00558181],
       [ 0.00759596,  0.02077762,  0.00807683, ..., -0.00074071,
        -0.01054709, -0.02763815],
       [ 0.00186106,  0.00201211,  0.01778391, ...,  0.00169023,
        -0.0117203 ,  0.00362528]])

In [97]:
X_train.shape

(80, 4096)

In [98]:
# getting the eigen faces
eigen_faces = np.dot(X_train, np.transpose(EigenVectors))

In [99]:
# for testing data => already done mean normalization
projected_data = np.dot(X_test, np.transpose(EigenVectors))

In [100]:
X_train = eigen_faces
X_test = projected_data

In [101]:
#predictions
predictions = []

for i in range(X_test.shape[0]) :
    min_distance = -1
    for j in range(X_train.shape[0]) :
        distance = np.linalg.norm(X_train[j] - X_test[i]) 
        if(min_distance == -1 or min_distance > distance) :
            min_distance = distance
            label = Y_train[j]
    predictions.append(label)


In [102]:
predictions

[1, 1, 6, 3, 2, 3, 3, 3, 4, 5, 5, 4, 5, 4, 6, 7, 7, 8, 9, 10]

In [103]:
Y_test

array([ 1,  1,  2,  2,  2,  3,  3,  3,  4,  5,  5,  5,  5,  5,  6,  7,  7,
        8,  9, 10])

In [104]:
#actual v/s prediction
for i in range(X_test.shape[0]):
    print("actual: " +Yname[Y_test[i]] +" , predictions: " + Yname[predictions[i]])

actual: Arpit , predictions: Arpit
actual: Arpit , predictions: Arpit
actual: Divyansh , predictions: Saumya
actual: Divyansh , predictions: Ishan
actual: Divyansh , predictions: Divyansh
actual: Ishan , predictions: Ishan
actual: Ishan , predictions: Ishan
actual: Ishan , predictions: Ishan
actual: Ishita , predictions: Ishita
actual: Ritik , predictions: Ritik
actual: Ritik , predictions: Ritik
actual: Ritik , predictions: Ishita
actual: Ritik , predictions: Ritik
actual: Ritik , predictions: Ishita
actual: Saumya , predictions: Saumya
actual: shantam , predictions: shantam
actual: shantam , predictions: shantam
actual: shashwat , predictions: shashwat
actual: Shivansh , predictions: Shivansh
actual: yash , predictions: yash


In [105]:
correct=0
for i in range(X_test.shape[0]):
    if predictions[i]==Y_test[i]:
        correct+=1
print("correct Predictions: " ,(correct) , " Out of: ",(X_test.shape[0]))
print("percentage accuracy: ", (correct / X_test.shape[0]) * 100)

correct Predictions:  16  Out of:  20
percentage accuracy:  80.0
